In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


with open('sherlock-holmes_stories_plain-text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [3]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [5]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [6]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [7]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [8]:


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)



Epoch 1/100
3010/3010 [==============================] - 180s 59ms/step - loss: 6.2288 - accuracy: 0.0775
Epoch 2/100
3010/3010 [==============================] - 216s 72ms/step - loss: 5.4862 - accuracy: 0.1270
Epoch 3/100
3010/3010 [==============================] - 242s 80ms/step - loss: 5.1049 - accuracy: 0.1496
Epoch 4/100
3010/3010 [==============================] - 191s 63ms/step - loss: 4.7809 - accuracy: 0.1664
Epoch 5/100
3010/3010 [==============================] - 186s 62ms/step - loss: 4.4802 - accuracy: 0.1830
Epoch 6/100
3010/3010 [==============================] - 167s 55ms/step - loss: 4.1936 - accuracy: 0.2026
Epoch 7/100
3010/3010 [==============================] - 169s 56ms/step - loss: 3.9220 - accuracy: 0.2265
Epoch 8/100
3010/3010 [==============================] - 192s 64ms/step - loss: 3.6653 - accuracy: 0.2576
Epoch 9/100
3010/3010 [==============================] - 182s 61ms/step - loss: 3.4244 - accuracy: 0.2898
Epoch 10/100
3010/3010 [======================

In [18]:

def generate_text(seed_text, next_words, tokenizer, model, max_sequence_len=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)


In [19]:

seed_text = "I will leave if they have"
next_words = 4


generate_text(seed_text, next_words, tokenizer, model, max_sequence_len)

1/1 [==============================] - 0s 21ms/step
I will leave if they have encountered any mysterious clues.
